In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import os
import pandas as pd
from tqdm import tqdm, trange
from collections import defaultdict
import itertools
import matplotlib.pyplot as plt
from networks import GGDRiskFieldNetwork
from torch.utils.data import DataLoader, TensorDataset

In [2]:
data = torch.load(r"D:\Productivity\Projects\RiskFieldImitation-main\data\redux-0218-2025\reduced_data")

In [4]:
neighbors = torch.load(r"D:\Productivity\Projects\RiskFieldImitation-main\data\redux-0218-2025\reduced_neighbor_indices")

In [6]:
neighbors.shape, data.shape

(torch.Size([954561415]), torch.Size([1208443, 10]))

tensor([[ 0.6589,  0.7455, -1.8949, -0.8720],
        [-1.7167,  1.2408, -0.1552, -0.8830]])

In [4]:
ggd = GGDRiskFieldNetwork(  input_dim=data.shape[1]-2,
                            name='ggd',
                        )

In [5]:
gamma = ggd.forward(data[:,:8])
gx = gamma[:,0]
gy = gamma[:,1]
x = data[:,8]
y = data[:,9]

In [22]:
k = 131
x = data[neighbors_dict[k]][:,8]
y = data[neighbors_dict[k]][:,9]
x.shape, y.shape

(torch.Size([822]), torch.Size([822]))

In [7]:
neighbor_loader = DataLoader(neighbors_dict, batch_size=256, shuffle=True)

In [12]:
len(neighbor_loader.dataset)

1208443

In [ ]:
all_neighbors = torch.cat(neighbors_dict) 

In [8]:
N = len(data)

In [9]:
query_indices = torch.repeat_interleave(torch.arange(N, device="cuda"), torch.tensor([len(n) for n in neighbors_dict], device="cuda"))

In [28]:
prob = [ torch.sum(1 + 1e-4 - torch.exp(-torch.abs(data[neighbors_dict[k]][:,8]/gx[k])**5-torch.abs(data[neighbors_dict[k]][:,9]/gy[k])**5)) for k in trange(len(data)) ]

 18%|█▊        | 214426/1208443 [00:40<03:08, 5272.83it/s]


KeyboardInterrupt: 

In [15]:
log = torch.log(prob)

In [ ]:
ux = torch.autograd.grad(log, gx,
                            create_graph=True,
                            allow_unused=True,
                            grad_outputs=torch.ones_like(log)
                          )[0]


uy = torch.autograd.grad(log, gy,
                            create_graph=True,
                            allow_unused=True,
                            grad_outputs=torch.ones_like(log)
                          )[0]
ux, uy